In [2]:
import pandas as pd
import numpy as np

##### Importing M11_Data from GitHub Repository

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/NNikhil89/TidyGrouping/main/M11_Data.csv")

In [4]:
df.head()

,Month,Category,Caltex,Gulf,Mobil
0,Open,Engine Oil,140 : 000,199 : 000,141 : 000
1,NaN,GearBox Oil,198 : 000,132 : 000,121 : 000
2,Jan,Engine Oil,170 : 103,194 : 132,109 : 127
3,NaN,GearBox Oil,132 : 106,125 : 105,191 : 100
4,Feb,Engine Oil,112 : 133,138 : 113,171 : 101


The given dataset is not in a long format because of below reasons:
* Column headers are values instead of variable names, here Column names Caltex, Gulf, Mobil which indicates different suppliers of oil are given as column names.
* Data values for multiple variables are contained within a single column, the data values for Caltex, Gulf and Mobil columns which are given in the format production:consumption denotes two indicators, oil production and oil consumption but are present in a single data point.
* Data values for variables are found within both rows and columns instead of being confined to a single column. In the given dataset, oil supplier values are given as column names, and production, consumption values are present as a single entry  in the dataframe.

Therefore in order to convert the given dataframe to long format: 
* we can use melt method of pandas where in we convert the column names Caltex, Gulf, Mobil as records in the dataframe also we can give Supplier as the name of this newly created column.
* The other change that we can perform is to split the data values present in the format production:consumption into two different columns Production and Consumption.
* This makes our dataset to contain a total of five columns Month, Category, Supplier, Production and Consumption.
* We can fill the missing values using ffill method as the missing values are clearly identified in the given dataset.


In [6]:
df_melt = pd.melt(df, id_vars=["Month","Category"], value_name="Produce_Consume", var_name="Supplier")

In [7]:
df_melt.head()

,Month,Category,Supplier,Produce_Consume
0,Open,Engine Oil,Caltex,140 : 000
1,NaN,GearBox Oil,Caltex,198 : 000
2,Jan,Engine Oil,Caltex,170 : 103
3,NaN,GearBox Oil,Caltex,132 : 106
4,Feb,Engine Oil,Caltex,112 : 133


* The above melted dataframe shows that the Supplier column names are converetd as data points, now we need to separate the Produce_Consume data values to two separate columns which indicates Production and Consumption of Oil.

* We use Regular expression (\d+)\s:\s+(\d+) to separate the column values as the Production and Consumption values are separated by :

In [9]:
tmp_df = df_melt["Produce_Consume"].str.extract("(\d+)\s:\s+(\d+)").astype(int)  
tmp_df.head()

,0,1
0,140,0
1,198,0
2,170,103
3,132,106
4,112,133


* A new dataframe tmp_df has been created with two columns which indicate the Production and Consumption of oil
* We will replace the default column names with Production and Consumption to give proper names to the columns

In [10]:
tmp_df.rename(columns={0:"Production", 1:"Consumption"}, inplace=True)

In [11]:
tmp_df.head()

,Production,Consumption
0,140,0
1,198,0
2,170,103
3,132,106
4,112,133


* The next step is to concatenate the tmp_df dataframe with df_melt dataframe

In [12]:
df_melt = pd.concat([df_melt, tmp_df], axis=1)

In [13]:
df_melt.head()

,Month,Category,Supplier,Produce_Consume,Production,Consumption
0,Open,Engine Oil,Caltex,140 : 000,140,0
1,NaN,GearBox Oil,Caltex,198 : 000,198,0
2,Jan,Engine Oil,Caltex,170 : 103,170,103
3,NaN,GearBox Oil,Caltex,132 : 106,132,106
4,Feb,Engine Oil,Caltex,112 : 133,112,133


* From the above resultant df_melt dataframe, we can remove the column Produce_Consume to transform it into tidy long format.

In [14]:
df_melt = df_melt.drop(["Produce_Consume"], axis=1)

In [15]:
df_melt.head() 

,Month,Category,Supplier,Production,Consumption
0,Open,Engine Oil,Caltex,140,0
1,NaN,GearBox Oil,Caltex,198,0
2,Jan,Engine Oil,Caltex,170,103
3,NaN,GearBox Oil,Caltex,132,106
4,Feb,Engine Oil,Caltex,112,133


* The Month column in df_melt dataframe has null values but on observing the pattern in the data, we can conclude that the missing month values are from its preceding row value in month column since the missing values are present only for GearBox oil category but for the same supplier we have month value for Engine oil category

In [17]:
df_melt['Month'].fillna(method='ffill', inplace = True)

In [96]:
df_melt.head()

,Month,Category,Supplier,Production,Consumption
0,Open,Engine Oil,Caltex,140,0
1,Open,GearBox Oil,Caltex,198,0
2,Jan,Engine Oil,Caltex,170,103
3,Jan,GearBox Oil,Caltex,132,106
4,Feb,Engine Oil,Caltex,112,133


* The resultant df_melt dataframe formed after melting, splitting multiple observations and filling missing values is tidy and in long format

* In order to find the amount of oil remaining for each type/brand we need to group by on Supplier and Category and then find the remaining oil for each supplier by finding the difference of sum of Production and Consumption for each supplier

* Grouping by Supplier and Category

In [19]:
grouped = df_melt.groupby(["Supplier","Category"])

* Considering Production and Consumption columns from DataFrameGroupBy object

In [22]:
grouped_ProdConsume = grouped[['Production', 'Consumption']]

* Performing sum aggregate operation on the grouped_ProdConsume object and then converting it to DataFrameobject

In [70]:
# Converting to DataFrame and assigning to dfAggSumProdConsume
dfAggSumProdConsume = pd.DataFrame(grouped_ProdConsume.agg(['sum']))
dfAggSumProdConsume

Production Consumption
                            sum         sum
Supplier Category                          
Caltex   Engine Oil        1084         754
         GearBox Oil       1121         746
Gulf     Engine Oil        1063         761
         GearBox Oil       1152         733
Mobil    Engine Oil        1021         676
         GearBox Oil       1020         721

* Calculate the amount of oil remaining by considering difference between aggregated Production and Consumption columns and assign the output to dfRemaining DataFrame

In [71]:
dfRemaining = dfAggSumProdConsume["Production"] - dfAggSumProdConsume["Consumption"]

* Renaming the column as Remaining

In [72]:
dfRemaining.rename(columns={'sum': 'Remaining'}, inplace=True)
dfRemaining

Remaining
Supplier Category              
Caltex   Engine Oil         330
         GearBox Oil        375
Gulf     Engine Oil         302
         GearBox Oil        419
Mobil    Engine Oil         345
         GearBox Oil        299

* The above result gives the amount of oil remaining for each type/brand at the end of the chronological period

* To find the most consumed brand of oil across the two separate categories/types of oil, we will group by df_melt dataframe on Category and Supplier and then aggregate the Consumption column by performing sum operation
* The above step gives us the total consumption of oil for each category and supplier
* Now to find the maximum oil consumption for each category, we wil reset the indexes of the groupbyobject obtained in first step and perform group by on category and select the row with maximum amount of consumption for each category.

In [80]:
#Perform group by on Category and Supplier and perform sum operation on the grouped object
groupedCategorySupp = df_melt.groupby(['Category', 'Supplier'])['Consumption'].sum()

#Reset the indexes of the above groupby object and store it in dataframe
dfmaxCategoryConsume = groupedCategorySupp.reset_index()

# Perform Group by Category on dfmaxCategoryConsume and find the row with the maximum Consumption for each category
maxSupplierCategory = dfmaxCategoryConsume.loc[dfmaxCategoryConsume.groupby('Category')['Consumption'].idxmax()]
maxSupplierCategory

,Category,Supplier,Consumption
1,Engine Oil,Gulf,761
3,GearBox Oil,Caltex,746


From the above result we can infer the following
* For the Engine Oil category, Gulf supplier had the maximum consumption with value of 761
* For the GearBox Oil category, Caltex supplier had the maximum consumption with value of 746

To transform a long format tidy data into wide format, we need to make certain adjustments to the visual presentation to make it easier to understand and interpret the data.
We can use hierarchical indexing or multi-level columns in the wide format to represent the different levels of information. 
We can mimic the structure of the given graphic by making adjustments to the given dataset.

Adjustments performed to convert to wide format:
* Need to make Month and Category columns as Indexes
* Transform the Supplier data values as Column names
* We can combine Production and Consumption data values into a single entity by joining them using a separator like :

#### Adjusting the Long format to make it suitable for Wide format

In [98]:
monthNames = ['Open', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']

# Converting Month column to categorical type
df_melt['Month'] = pd.Categorical(df_melt['Month'], categories=monthNames, ordered=True)

# Aggregating Production and Consumption Columns and assigning it to a new dataframe
df_wide = df_melt.groupby(['Month', 'Category', 'Supplier']).agg({'Production': 'sum', 'Consumption': 'sum'}).reset_index()

# Combining Production and Consumption columns
df_wide['Prod_Consume'] = df_wide['Production'].astype(str) + ' : ' + df_wide['Consumption'].astype(str)

# Drop the Production and Consumption columns
df_wide.drop(['Production', 'Consumption'], axis=1, inplace=True)

df_wide.head()

,Month,Category,Supplier,Prod_Consume
0,Open,Engine Oil,Caltex,140 : 0
1,Open,Engine Oil,Gulf,199 : 0
2,Open,Engine Oil,Mobil,141 : 0
3,Open,GearBox Oil,Caltex,198 : 0
4,Open,GearBox Oil,Gulf,132 : 0


#### Creating pivot table using the transformed dataframe to create the wide format which mimics the given graphic

In [99]:
# Creating Pivot table using indexes Month and Category
df_pivot = df_wide.pivot_table(index=['Month', 'Category'], columns='Supplier', values='Prod_Consume', aggfunc='first')
df_pivot


Supplier              Caltex       Gulf      Mobil
Month Category                                    
Open  Engine Oil     140 : 0    199 : 0    141 : 0
      GearBox Oil    198 : 0    132 : 0    121 : 0
Jan   Engine Oil   170 : 103  194 : 132  109 : 127
      GearBox Oil  132 : 106  125 : 105  191 : 100
Feb   Engine Oil   112 : 133  138 : 113  171 : 101
      GearBox Oil  193 : 148  199 : 119  134 : 127
Mar   Engine Oil   184 : 100  141 : 141  114 : 108
      GearBox Oil  138 : 121  172 : 133  193 : 115
Apr   Engine Oil   149 : 150  117 : 118  117 : 118
      GearBox Oil  185 : 125  191 : 133  119 : 121
May   Engine Oil   170 : 139  104 : 119  200 : 117
      GearBox Oil  168 : 117  138 : 102  121 : 146
Jun   Engine Oil   159 : 129  170 : 138  169 : 105
      GearBox Oil  107 : 129  195 : 141  141 : 112

### Loading Data

In [81]:
# load the data set 
auto_df = pd.read_csv("https://raw.githubusercontent.com/NNikhil89/TidyGrouping/main/auto-mpg.data", delim_whitespace = True, header = None)

In [82]:
auto_df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model', 'origin', 'car_name']
# replace '?' in horsepower column with 'NaN' 
auto_df.horsepower.replace('?', np.nan, inplace = True)
# convert the column to numeric 
auto_df["horsepower"] = pd.to_numeric(auto_df["horsepower"])
# replace origin values using a dict 
auto_df.origin.replace({1: 'USA', 2: 'Asia', 3: 'Europe'}, inplace = True)
auto_df.head(10)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,USA,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,USA,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,USA,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,USA,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,USA,ford torino
5,15.0,8,429.0,198.0,4341.0,10.0,70,USA,ford galaxie 500
6,14.0,8,454.0,220.0,4354.0,9.0,70,USA,chevrolet impala
7,14.0,8,440.0,215.0,4312.0,8.5,70,USA,plymouth fury iii
8,14.0,8,455.0,225.0,4425.0,10.0,70,USA,pontiac catalina
9,15.0,8,390.0,190.0,3850.0,8.5,70,USA,amc ambassador dpl


* For the given auto_df dataframe we will perform grouping on the columns origin, cylinders and then count the number of records for each group using size method and rename the resultant count column with Quantity
* Then convert the resultant grouped operation to a dataframe autodf which gives us the expected result.

In [85]:
# Perform the grouing operation on origin and cylinders and assign the resultant gropued operation to autodf dataframe
autodf = pd.DataFrame(auto_df.groupby(['origin','cylinders']).size().rename('Quantity'))
autodf

Quantity
origin cylinders          
Asia   4                63
       5                 3
       6                 4
Europe 3                 4
       4                69
       6                 6
USA    4                72
       6                74
       8               103

Steps followed to generate the report that shows the average miles per gallon:
* For the given auto_df dataframe we will perform grouping on the columns, origin and model.
* Consider mpg and weight columns from the grouped dataframe
* Perform aggregation mean operation on the columns mpg and weight
* Then convert the resultant grouped operation to a dataframe mpgWeightdf which gives us the expected result.

In [86]:
mpgWeightdf = pd.DataFrame(auto_df.groupby(['origin','model'])[['mpg','weight']].agg(['mean']))
mpgWeightdf

mpg       weight
                   mean         mean
origin model                        
Asia   70     25.200000  2309.200000
       71     28.750000  2024.000000
       72     22.000000  2573.200000
       73     24.000000  2335.714286
       74     27.000000  2139.333333
       75     24.500000  2571.166667
       76     24.250000  2611.000000
       77     29.250000  2138.750000
       78     24.950000  2691.666667
       79     30.450000  2693.750000
       80     37.288889  2348.000000
       81     31.575000  2725.000000
       82     40.000000  2055.000000
Europe 70     25.500000  2251.000000
       71     29.500000  1936.000000
       72     24.200000  2300.400000
       73     20.000000  2397.250000
       74     29.333333  2053.000000
       75     27.500000  2303.250000
       76     28.000000  2217.500000
       77     27.416667  2295.833333
       78     29.687500  2221.250000
       79     32.950000  1997.500000
       80     35.400000  2290.307692
       81     32.958333  2269.166667
       82     34.888889  2132.777778
USA    70     15.272727  3716.500000
       71     18.100000  3401.600000
       72     16.277778  3682.666667
       73     15.034483  3821.448276
       74     18.333333  3503.333333
       75     17.550000  3533.200000
       76     19.431818  3405.409091
       77     20.722222  3422.000000
       78     21.772727  3141.136364
       79     23.478261  3210.217391
       80     25.914286  2822.428571
       81     27.530769  2695.000000
       82     29.450000  2637.750000

### To generate report that shows the average weight, count, minimum weight, maximum weight, and median weight of vehicles manufactured during 5 equal length time periods.

Steps followed to generate the report that describes the weight column:
* For the given auto_df dataframe we will perform cut operation on model column with bins set to 5 and precision to 0, to obtain five quartiles.
* Then define a method get_Weight_Stats which accepts a grouped dataframe object as parameter and performs the Average Weight, Count, Maximum Weight, Median Weight and Minimum weight opeartions on the input parameter and returns the result containing the performed operations in the form of dictionary
* Next perform grouping on weight column of auto_df based on the obtained quartiles from step 1.
* Perform apply operation on the grouped object by sending the above defined method name get_Weight_Stats as input to apply function 
* Then convert the resultant operation to a dataframe weightStats which gives us the expected result.

In [100]:
quartiles = pd.cut(auto_df.model, bins=5, precision=0)

In [89]:
def get_Weight_Stats(group):
    return {'Average Weight': group.mean(), 'Count': group.count(),
            'Max Weight': group.max(), 'Median Weight': group.median(), 'Min Weight': group.min()}

# group the weight column's values relative to the model quartile cuts we made above 
# and apply the get_Weight_Stats function to the grouped weight values
weightStats = pd.DataFrame(auto_df.weight.groupby(pd.cut(auto_df.model, bins=5, precision=0)).apply(get_Weight_Stats))
weightStats

weight
model                                   
(70.0, 72.0] Average Weight  3203.988235
             Count             85.000000
             Max Weight      5140.000000
             Median Weight   3139.000000
             Min Weight      1613.000000
(72.0, 75.0] Average Weight  3200.970149
             Count             67.000000
             Max Weight      4997.000000
             Median Weight   2945.000000
             Min Weight      1649.000000
(75.0, 77.0] Average Weight  3085.945652
             Count             92.000000
             Max Weight      4668.000000
             Median Weight   3062.000000
             Min Weight      1795.000000
(77.0, 80.0] Average Weight  2948.153846
             Count             65.000000
             Max Weight      4360.000000
             Median Weight   2990.000000
             Min Weight      1800.000000
(80.0, 82.0] Average Weight  2470.651685
             Count             89.000000
             Max Weight      3725.000000
             Median Weight   2395.000000
             Min Weight      1755.000000